**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

```epsilon``` will allow the Agent to try things that are different from its current best policy, once in a while. Supposedly, ```espilon``` is quite small such that the agent will usually have some time to estimate the efficiency of its current actions and refine them before trying something new. Whenever the exploration probability ```epsilon``` is obtained, a random action is taken, which will likely lead to poorer results, but it is something essential in order for our agent to explore unknown actions and game states. It will then learn the impact of this random action and might discover something valuable doing so.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [13]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite("videos/" + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))
        b[self.board > 0, 0] = 128
        b[self.board > 0, 1] = 128
        b[self.board < 0, 0] = 96
        b[self.board < 0, 2] = 128
        b[self.x,self.y,:]=128
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        b[self.x-2:self.x+3,self.y-2:self.y+3] *= 2
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [14]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open("videos/" + name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` contains the bonuses and maluses of the island: cheese and poison. As we are exploring it, these disappear (replaced by 0) and given as the reward.

The array ```position``` is initialized full of zeros and later on represents the last position of the rat with a 1 at the corresponding location. It gives us the information of where we were last turn. It is unclear though how the network will know where we are at the current state, but it might learn that on itself.

Both arrays have 2 more cells on each side to avoid any bug with visibility distance, but those supplementary cells are cut off when returning the current state to the Agent. Moreover, if the agent is trying to move towards the limit of the grid, the move will happen in the opposite direction as if it just bounced against a wall.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [15]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return random.randint(0, 3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [16]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # Start a new game
        state = env.reset()
        game_over = False
        
        # Score counters
        win = 0
        lose = 0

        while not game_over:
            # Retrieve the action without exploring with espilon
            action = agent.act(state, train=False)

            # Apply the action to the environnement
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            
            # -- We don't reinforce as we are testing

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [18]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/15.0. Average score (-5.0)
Win/lose count 8.5/18.0. Average score (-7.25)
Win/lose count 5.0/10.0. Average score (-6.5)
Win/lose count 9.5/19.0. Average score (-7.25)
Win/lose count 11.5/16.0. Average score (-6.7)
Final score: -6.7


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




#### Answer:

We start from the definition of $Q^\pi(s,a)$, letting $t$ evolve towards infinity:
\begin{align*}
    Q^\pi(s,a) &= E_{p^\pi}\left[\sum_{t=0}^\infty\gamma^t r(s_t,a_t)|s_0=s,a_0=a\right] && \text{}\\
    Q^\pi(s,a) &= \sum_{t=0}^\infty E_{p^\pi}\left[\gamma^t r(s_t,a_t)|s_0=s,a_0=a\right] && \text{Linearity of the expected value.}\\
    Q^\pi(s,a) &= E_{p^\pi}\left[r(s_0,a_0)|s_0=s,a_0=a\right] + \sum_{t=1}^\infty E_{p^\pi}\left[\gamma^t r(s_t,a_t)|s_0=s,a_0=a\right] && \text{Extract first term.}\\
    Q^\pi(s,a) &= r(s,a) + \sum_{t=1}^\infty E_{p^\pi}\left[\gamma^t r(s_t,a_t)|s_0=s,a_0=a\right] && \text{First term is known.}\\
    Q^\pi(s,a) &= r(s,a) + E_{(s',a')\sim p(.|s,a)} \left[\sum_{t=1}^\infty E_{p^\pi}\left[\gamma^t r(s_t,a_t)|s_1=s',a_1=a'\right]\right] && \text{Joint distribution over s' and a'.}\\
    Q^\pi(s,a) &= r(s,a) + E_{(s',a')\sim p(.|s,a)} \left[\gamma\sum_{t=0}^\infty E_{p^\pi}\left[\gamma^t r(s_t,a_t)|s_0=s',a_0=a'\right]\right] && \text{Change of index and getting $\gamma$ out.}\\
    Q^\pi(s,a) &= r(s,a) + E_{(s',a')\sim p(.|s,a)} \left[\gamma Q^\pi(s',a')\right] && \text{We find $Q$ expression for $s'$ and $a'$.}\\
    Q^\pi(s,a) &= E_{(s',a')\sim p(.|s,a)} \left[r(s,a) + \gamma Q^\pi(s',a')\right] && \text{Linearity of the expected value.}
\end{align*}

The optimal policy $\pi^*$ consists in always choosing the action which leads to the greatest reward $R$, hence has an optimal $Q$:
\begin{align*}
    Q^*(s,a) &= Q^{\pi^*}(s,a)\\
    Q^*(s,a) &= E_{(s',a')\sim \pi^*(.|s,a)} \left[r(s,a) + \gamma Q^{\pi^*}(s',a')\right] && \text{By applying the above.}\\
    Q^*(s,a) &= E_{s'\sim \pi^*(.|s,a)} \left[r(s,a) + \gamma \max_{a'} Q^*(s',a')\right] && \text{By independence to $a'$ as we always maximise $Q$}
\end{align*}

We can now start from an intuitively desirable loss function:
\begin{align*}
    \mathcal{L}^*(\theta) &= ||Q^*(s,a) - Q(s,a,\theta)||^2 && \text{We want to get closer to the optimal $Q$.}\\
    \mathcal{L}^*(\theta) &= ||E_{s'\sim \pi^*(.|s,a)} \left[r(s,a) + \gamma \max_{a'} Q^*(s',a')\right] - Q(s,a,\theta)||^2 && \text{By applying the above.}\\
    \mathcal{L}^*(\theta) &= E_{s'\sim \pi^*(.|s,a)} \left[r(s,a) + \gamma \max_{a'} Q^*(s',a') - Q(s,a,\theta)\right]^2 && \text{By independence to $a$ and $s$.}\\
    \mathcal{L}(\theta) &= E_{s' \sim \pi^*(.|s,a)} \left[r(s,a)+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\right]^{2} && \text{We define the realistic loss by using our real $Q$.}
\end{align*}
There is a sense in replacing $Q^*$ in the previous equation by our modellized $Q$ because we are applying it on the next state to compare it to the previous step value plus the real reward we obtained at this turn. It turns out that the two $Q$ tend to cancel eachother except for the reward of the current turn: $r$. This means that we are in fact optimizing our knowledge of the reward following an action.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [19]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [20]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # Start a new game
        state = env.reset()
        game_over = False

        # Score counters
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score = 0.9 * score + 0.1 * (win-lose)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Averaged: {}"
              .format(e, epoch, loss, win, lose, win-lose, score))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [21]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1, 5, 5, self.n_state)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        target_q = np.zeros((self.batch_size, 4))
        actions = np.zeros((self.batch_size, 4))
        
        rewards = np.zeros((self.batch_size))
        gameovers = np.zeros((self.batch_size))
        for i in range(self.batch_size):
            s, ns, a, r, go = self.memory.random_access()
            input_states[i, :, :, :] = s
            next_states[i, :, :, :] = ns
            actions[i, a] = 1
            rewards[i] = r
            gameovers[i] = go
        
        target_q[:, :] = self.model.predict(input_states)
        target_q[actions == 1] = rewards + self.discount * np.max(self.model.predict(next_states), axis=1) * (1 - gameovers)
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([
            keras.layers.Flatten(input_shape=(5, 5, self.n_state)),
            Dense(128, activation="relu"),
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(32, activation="relu"),
            Dense(32, activation="relu"),
            Dense(32, activation="relu"),
            Dense(4)
        ])
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")

        self.model = model
        

In [23]:
epochs_train = 101

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.05, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               6528      
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_19 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_20 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_21 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_22 (Dense)             (None, 32)               

Epoch 074/101 | Loss 0.0016 | Win/lose count 6.5/1.0 (5.5) | Averaged: 2.7105460209988204
Epoch 075/101 | Loss 0.0414 | Win/lose count 3.0/0 (3.0) | Averaged: 2.7394914188989388
Epoch 076/101 | Loss 0.0035 | Win/lose count 4.0/2.0 (2.0) | Averaged: 2.6655422770090453
Epoch 077/101 | Loss 0.0021 | Win/lose count 7.0/1.0 (6.0) | Averaged: 2.998988049308141
Epoch 078/101 | Loss 0.0026 | Win/lose count 3.0/2.0 (1.0) | Averaged: 2.7990892443773268
Epoch 079/101 | Loss 0.0018 | Win/lose count 5.5/0 (5.5) | Averaged: 3.069180319939594
Epoch 080/101 | Loss 0.0034 | Win/lose count 14.0/1.0 (13.0) | Averaged: 4.062262287945635
Epoch 081/101 | Loss 0.0018 | Win/lose count 8.0/1.0 (7.0) | Averaged: 4.356036059151071
Epoch 082/101 | Loss 0.0028 | Win/lose count 9.5/3.0 (6.5) | Averaged: 4.570432453235965
Epoch 083/101 | Loss 0.0409 | Win/lose count 1.5/0 (1.5) | Averaged: 4.263389207912368
Epoch 084/101 | Loss 0.0018 | Win/lose count 7.5/0 (7.5) | Averaged: 4.587050287121132
Epoch 085/101 | Loss 0.

In [24]:
HTML(display_videos('fc_train100.mp4'))

After some tuning, I quickly found a set of parameters (learning rate, batch size...) that lead to improving strategies during training. To choose a great neural network architecture, I applied some recommendations from Deep Learning in Practice course and don't really bother about putting too much layers or neurons in layers. I tried at first with a smaller architecture but noticed a slight improvement by increasing the number of layers, so I kept it that way.
As the next question suggests, I give more credit to CNN in this specific application so I didn't spend too much time fine-tuning this DQN network.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [25]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
            Conv2D(32, 3, input_shape=(5, 5, self.n_state), activation="relu"),
            Conv2D(32, 3, activation="relu"),
            keras.layers.Flatten(),
            Dense(128, activation="sigmoid"),
            Dense(4)
        ])
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [26]:
epochs_train = 101

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 32)          608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_4 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 516       
Total params: 14,596
Trainable params: 14,596
Non-trainable params: 0
_________________________________________________________________
Epoch 000/101 | Loss 0.0021 | Win/lose count 4.0/5.0 (-1.0) | Averaged: -0.1
Epoch 001/101 | Loss 0.0020 | Win/lose 

Epoch 080/101 | Loss 0.0012 | Win/lose count 1.0/2.0 (-1.0) | Averaged: 10.514409892729356
Epoch 081/101 | Loss 0.0035 | Win/lose count 16.5/2.0 (14.5) | Averaged: 10.91296890345642
Epoch 082/101 | Loss 0.0016 | Win/lose count 13.5/1.0 (12.5) | Averaged: 11.071672013110778
Epoch 083/101 | Loss 0.0017 | Win/lose count 8.0/0 (8.0) | Averaged: 10.764504811799702
Epoch 084/101 | Loss 0.0013 | Win/lose count 17.0/4.0 (13.0) | Averaged: 10.988054330619732
Epoch 085/101 | Loss 0.0026 | Win/lose count 14.5/3.0 (11.5) | Averaged: 11.03924889755776
Epoch 086/101 | Loss 0.0014 | Win/lose count 22.0/1.0 (21.0) | Averaged: 12.035324007801984
Epoch 087/101 | Loss 0.0016 | Win/lose count 13.5/1.0 (12.5) | Averaged: 12.081791607021787
Epoch 088/101 | Loss 0.0019 | Win/lose count 11.5/0 (11.5) | Averaged: 12.023612446319609
Epoch 089/101 | Loss 0.0012 | Win/lose count 16.0/2.0 (14.0) | Averaged: 12.221251201687648
Epoch 090/101 | Loss 0.0013 | Win/lose count 14.5/1.0 (13.5) | Averaged: 12.3491260815188

In [27]:
HTML(display_videos('cnn_train100.mp4'))

Compared to DQN's training, breakthroughs are happening twice sooner in the CNN's training and we are finally reaching an average score twice higher after 100 epochs. It seems this architecture is more adapted to our environnement: intuitively, if the CNN learns something happening on the bottom right corner of the visible space, it will also be learnt for the top left corner (at least concerning the two first layers which are convolutionnal, hence for feature extraction).

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [28]:
epochs_test = 11

env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 3, 32)          608       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_5 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 516       
Total params: 14,596
Trainable params: 14,596
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                

In [31]:
HTML(display_videos('cnn_test1.mp4'))

In [30]:
HTML(display_videos('fc_test1.mp4'))

Quite obviously, we see an undesirable behavior where both DQN and CNN are getting stuck in endless loops after a few moves. This happens when there is a cycle between two steps for which the maximum policy action leads to each other. During training, we didn't see this because there was a 10% chance of doing a random move, which would get the agent out of the loop quite fast.

This is a problem we adress in the next questions with a more sophisticated version of the game (which could be performed entirely from the agent's perspective, we are not modifying the rules of the game).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [32]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # Start a new game
        state = env.reset()
        game_over = False

        # Score counters
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score = 0.9 * score + 0.1 * (win-lose)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Averaged: {}"
              .format(e, epoch, loss, win, lose, win-lose, score))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.explored = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite("videos/" + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        state = np.zeros_like(self.board)
        
        b = np.zeros((self.grid_size,self.grid_size,3))
        b[self.board > 0, 0] = 128
        b[self.board > 0, 1] = 128
        b[self.board < 0, 0] = 96
        b[self.board < 0, 2] = 128
        b[self.x,self.y,:]=128
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        b[:, :, 0] += self.explored[:, :] * 96
        b[self.x-2:self.x+3,self.y-2:self.y+3] *= 2
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        self.explored *= 0.99
        self.explored[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y] - ((self.explored[self.x, self.y] * 0.2) if train else 0)
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.explored.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.explored = np.zeros((self.grid_size, self.grid_size))
#         self.explored[0:2,:]= -1
#         self.explored[:,0:2] = -1
#         self.explored[-2:, :] = -1
#         self.explored[-2:, :] = -1
        
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                        self.board.reshape(self.grid_size, self.grid_size, 1),
                        self.position.reshape(self.grid_size, self.grid_size, 1),
                        self.explored.reshape(self.grid_size, self.grid_size, 1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [33]:
# Training
epochs_train = 101

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
agent.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')

train_explore(agent, env, epochs_train, prefix='cnn_train_explore')

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 3, 3, 32)          896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_7 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_38 (Dense)             (None, 4)                 516       
Total params: 14,884
Trainable params: 14,884
Non-trainable params: 0
_________________________________________________________________
Epoch 000/101 | Loss 0.0359 | Win/lose count 22.5/23.66489781046136 (-1.1648978104613583) | Averaged: -0.11648978104

Epoch 061/101 | Loss 0.0280 | Win/lose count 23.5/14.007812919185636 (9.492187080814364) | Averaged: 6.263329400761728
Epoch 062/101 | Loss 0.0323 | Win/lose count 29.0/14.888362324688696 (14.111637675311304) | Averaged: 7.048160228216686
Epoch 063/101 | Loss 0.0333 | Win/lose count 21.5/19.389611938397806 (2.110388061602194) | Averaged: 6.554383011555237
Epoch 064/101 | Loss 0.0198 | Win/lose count 25.0/20.53420378479485 (4.465796215205149) | Averaged: 6.345524331920228
Epoch 065/101 | Loss 0.0135 | Win/lose count 23.5/20.580821051494848 (2.9191789485051522) | Averaged: 6.002889793578721
Epoch 066/101 | Loss 0.0255 | Win/lose count 18.5/13.587509174153197 (4.9124908258468025) | Averaged: 5.893849896805529
Epoch 067/101 | Loss 0.0307 | Win/lose count 28.0/18.69841163818374 (9.301588361816261) | Averaged: 6.234623743306603
Epoch 068/101 | Loss 0.0217 | Win/lose count 27.5/14.476832539026553 (13.023167460973447) | Averaged: 6.913478115073288
Epoch 069/101 | Loss 0.0141 | Win/lose count 2

In [34]:
# Evaluation
epochs_test = 1
env = EnvironmentExploring(grid_size=25, max_time=1000, temperature=0.4)
test(agent,env,epochs_test,prefix='cnn_test_explore')

Win/lose count 118.0/7.0. Average score (111.0)
Final score: 111.0


In [35]:
HTML(display_videos('cnn_test_explore0.mp4'))

To see a greater chunk of behavior, I tested the last agent on a bigger map and a longer game. I also added a visualization of the improvement I made, following the question recommendations.

The idea was to remember the explored cells on a separate grid, as if the visited cell were getting "hot", but then decrease the heat value of previously visited cells at each turn, multiplying the matrix by a constant (0.98 here). You can see the hot cells in red in the visualisation.

The interest of remembering explored cells is to give a malus to the agent whenever it is revisiting the same cells again, forcing it to explore new regions of the map.

The reason behind the heat decreasing was to avoid the endless loops that were still persisting even with this modification. As heat decreases, the perceived state changes and after a few turns it becomes more interesting for the agent to get away (surrounding cells become more interesting that cells it is looping on).

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***